In [4]:
#jupyter nbextension enable --py gmaps

In [3]:
# Dependencies
from census import Census
from config import (ow_key, gkey)
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt

In [10]:
file = "../WeatherPy/weather-data.csv"
weather_data_df = pd.read_csv(file, encoding="utf-8")
weather_data_df.head()

,City,Latitude,Degrees From Equator,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,albany,42.6001,42.6001,-73.9662,4.0,41.0,5.0,4.47
1,cape town,-33.9258,33.9258,18.4232,22.0,56.0,0.0,5.66
2,tombouctou,20.0000,20.0000,-3.0000,39.0,5.0,1.0,1.54
3,olafsvik,64.8945,64.8945,-23.7142,6.0,90.0,100.0,6.21
4,bowmore,34.9421,34.9421,-79.3031,7.0,65.0,1.0,2.06


In [11]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)


In [19]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_data_df[["Latitude", "Longitude"]][0:5]

humidity = weather_data_df["Humidity"][0:5]

#locations.head()
humidity

0    41.0
1    56.0
2     5.0
3    90.0
4    65.0
Name: Humidity, dtype: float64

In [21]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)


fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [37]:
ideal_vacay_spots = weather_data_df.loc[(weather_data_df['Temperature'] >= 22) &  
                                        (weather_data_df['Temperature'] <= 25) &
                                        (weather_data_df['Wind Speed'] <= 5) & 
                                        (weather_data_df['Cloudiness'] == 0)
                                        ].reset_index(drop=True)

ideal_vacay_spots

,City,Latitude,Degrees From Equator,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,sao joao da barra,-21.6403,21.6403,-41.0511,23.0,68.0,0.0,2.57
1,saldanha,-33.0117,33.0117,17.9442,23.0,70.0,0.0,4.12
2,gat,31.6100,31.6100,34.7642,24.0,50.0,0.0,0.89
3,vilhena,-12.7406,12.7406,-60.1458,24.0,88.0,0.0,3.09
4,prata,-19.3072,19.3072,-48.9242,25.0,41.0,0.0,2.26
5,fez,34.0372,34.0372,-4.9998,25.0,44.0,0.0,4.12
6,santo antonio do monte,-20.0872,20.0872,-45.2936,23.0,52.0,0.0,0.45
7,kuito,-12.3833,12.3833,16.9333,23.0,56.0,0.0,3.09


In [ ]:
Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.


Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.